In [1]:
import sys
sys.path.insert(0, '..')
%load_ext autoreload
%autoreload 2
%aimport std_func

# Hide warnings
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pattern'

## Estimates from Factor Model
We assume a factor model where there are k assets and T time periods (months in our case), m is the common factors (topic weights in our case). $r_{it}$ is the return for asset $i$ at time $t$.

$$
\begin{align}
r_{it}
= &\alpha_i + \sum_{j=1}^{m} \beta_{ij} f_jt + \epsilon_{it}, t = 1, \dots, T, i = 1, \dots, k\\ \\
R_{k \times T} = & B_{k \times m} \cdot coef_{m \times T} + E_{k \times T}
\end{align}
$$

In our analysis, $R_{k \times T}$ is the returns matrix imported, $B_{k \times m}$ is the topic modeling matrix, and $coef_{m \times T}$ is the coefficient matrix computed after the linear regression on returns matrix and topic modeling matrix. $E_{k \times T}$ is the residual matrix.

In our factor model, 

$$\text{Cov}(r) = B \Sigma_{coef} B^T + D, 
\text{ where } D = diag(\sigma^2_1, \dots, \sigma^2_k) \text{ and Var}(\epsilon_i) = \sigma^2_i$$

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import json
import string
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
r_selected = pd.read_csv("data/filtered_r.csv")
# get the mean of all 
r_selected.set_index("name", inplace = True)
mu = r_selected.mean(axis = 1)
# compute the covariance matrix 
cov = r_selected.T.cov()

In [4]:
df = pd.read_csv('../data/preprocessed.csv',
                 usecols = ['reportingDate', 'name', 'CIK', 'coDescription',
                           'coDescription_stopwords', 'SIC', 'SIC_desc'])
df = df.set_index(df.name)

### Sent-LDA
We ran the coherence score benchmarking over a range of 3 to 40 topics, incrementing by 3. 

First, we fit the LDA model to all business description using the number of topics selected from coherence score benchmarking. 

Then, we assume each sentence only represents one topic; get the frequency of the topics revealed in the whole document (business description for one company) and calculate the probability of each topics in the whole document. 

#### Coherence Score Plot

![Factor_Model_Coherence_Score.png](images/Factor_Model_Coherence_Score.png)

Based on the above Coherence Score, we choose up to 12 topics since it gives the highest score up to here.

In [5]:
data = df.loc[:,"coDescription_stopwords"].to_list()

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.85, min_df=2, max_features=600)
tf = tf_vectorizer.fit_transform(data)
tf_feature_names = tf_vectorizer.get_feature_names_out()
lda = LatentDirichletAllocation(n_components=12, random_state=0).fit(tf)

We show the top 10 words by weights in the 12 topics LDA model generates in the below table.

In [7]:
std_func.get_topics(lda, tf_vectorizer, 12)

NameError: name 'std_func' is not defined

#### Frequency of the Topics in Each Sentence

In [8]:
n_components = 12
prob = pd.DataFrame(0, index = df.name, columns = range(n_components))

In [9]:
for j in range(len(df)):
    LIST_sent = pd.Series(df.coDescription[j].split('.')).apply(std_func.lemmatize_sentence).apply(std_func.remove_nums).apply(std_func.remove_stopwords)
    
    X = tf_vectorizer.transform(LIST_sent.tolist())
    sent = lda.transform(X)
    sent_df = pd.DataFrame(sent)
    # drop the values that are smaller than 1/12
    # if the maximum value is 1/12, the probability of each topic in that sentence is the same
    # we cannot determine which topic to choose
    sent_df = sent_df[sent_df.max(axis = 1) > 1/12].reset_index(drop = True)

    for i in range(n_components):
        prob.iloc[j][i] = list(sent_df.idxmax(axis = 1)).count(i)
    
    # calculate the probability
    prob = prob.div(prob.sum(axis=1), axis=0)

NameError: name 'std_func' is not defined

In [10]:
prob

,0,1,2,3,4,5,6,7,8,9,10,11
name,,,,,,,,,,,,
"MONGODB, INC.",0,0,0,0,0,0,0,0,0,0,0,0
SALESFORCE COM INC,0,0,0,0,0,0,0,0,0,0,0,0
SPLUNK INC,0,0,0,0,0,0,0,0,0,0,0,0
"OKTA, INC.",0,0,0,0,0,0,0,0,0,0,0,0
VEEVA SYSTEMS INC,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
"AMERICAN REALTY CAPITAL NEW YORK CITY REIT, INC.",0,0,0,0,0,0,0,0,0,0,0,0
"CYCLACEL PHARMACEUTICALS, INC.",0,0,0,0,0,0,0,0,0,0,0,0
ZOETIS INC.,0,0,0,0,0,0,0,0,0,0,0,0


#### Demonstration in Pharmaceutical Preparations Industry

In [11]:
# get the names of the companies in the pharmaceutical preparations industry
Pharm = df[df.SIC == 2834]
Pharm_list = Pharm.index

In [12]:
# get the companies name that match return data and business description data
SET = (set(Pharm_list) & set(r_selected.index))
LIST = [*SET, ]

In [13]:
B_matrix = prob.T[LIST].T
B_matrix = B_matrix[~B_matrix.index.duplicated(keep="first")]

##### Topic Matrix

In [14]:
B_matrix

,0,1,2,3,4,5,6,7,8,9,10,11
name,,,,,,,,,,,,
"RECRO PHARMA, INC.",0,0,0,0,0,0,0,0,0,0,0,0
"MIRATI THERAPEUTICS, INC.",0,0,0,0,0,0,0,0,0,0,0,0
"KEMPHARM, INC",0,0,0,0,0,0,0,0,0,0,0,0
"HERON THERAPEUTICS, INC. /DE/",0,0,0,0,0,0,0,0,0,0,0,0
ALEXION PHARMACEUTICALS INC,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
"SIERRA ONCOLOGY, INC.",0,0,0,0,0,0,0,0,0,0,0,0
"FIBROCELL SCIENCE, INC.",0,0,0,0,0,0,0,0,0,0,0,0
WAVE LIFE SCIENCES LTD.,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
r_Pharm = r_selected.T[LIST].T

In [16]:
coef_mat = pd.DataFrame(0, index = r_Pharm.columns, columns = range(n_components))

In [17]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression()
for i in range(len(r_Pharm.columns)):
    date = r_Pharm.columns[i]
    r_t_i = r_Pharm[date] 
    reg = LR.fit(B_matrix, r_t_i)
    #print(reg.score(B_matrix, r_t_i))
    coef_mat.iloc[i] = reg.coef_

##### Coefficient Matrix

In [18]:
coef_mat

,0,1,2,3,4,5,6,7,8,9,10,11
2016-06-30,0,0,0,0,0,0,0,0,0,0,0,0
2016-07-31,0,0,0,0,0,0,0,0,0,0,0,0
2016-08-31,0,0,0,0,0,0,0,0,0,0,0,0
2016-09-30,0,0,0,0,0,0,0,0,0,0,0,0
2016-10-31,0,0,0,0,0,0,0,0,0,0,0,0
2016-11-30,0,0,0,0,0,0,0,0,0,0,0,0
2016-12-31,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-31,0,0,0,0,0,0,0,0,0,0,0,0
2017-02-28,0,0,0,0,0,0,0,0,0,0,0,0
2017-03-31,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
prediction = B_matrix.dot(coef_mat.T)
residual = r_Pharm - prediction

### Perform Mean-Variance Analysis
We only use the Pharmaceutical Preparations industry data to generate portfolio based on Mean-Variance Analysis.

In [20]:
!pip install PyPortfolioOpt

In [21]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
from pypfopt import plotting

ModuleNotFoundError: No module named 'pypfopt'

#### Mean for Returns in the Pharmaceutical Preparations Industry

In [22]:
mu_Pharm = mu[LIST]
mu_Pharm

name
RECRO PHARMA, INC.               0.008656
MIRATI THERAPEUTICS, INC.        0.075113
KEMPHARM, INC                   -0.012939
HERON THERAPEUTICS, INC. /DE/    0.013116
ALEXION PHARMACEUTICALS INC     -0.007589
                                   ...   
SIERRA ONCOLOGY, INC.           -0.019358
FIBROCELL SCIENCE, INC.         -0.063871
WAVE LIFE SCIENCES LTD.          0.048065
LILLY ELI & CO                   0.017574
OPHTHOTECH CORP.                -0.076591
Length: 124, dtype: float64

#### Covariance Estimated
$\text{Cov}(r) = \beta \Sigma_{coef} \beta^T + D, 
\text{ where } D = diag(\sigma^2_1, \dots, \sigma^2_k)$

In [23]:
cov_Factor_Model = pd.DataFrame(np.array(B_matrix.dot(coef_mat.cov()).dot(B_matrix.T)) + np.diag(np.diag(residual.T.cov()))).set_index(B_matrix.index)
cov_Factor_Model.columns = B_matrix.index
cov_Factor_Model

name,"RECRO PHARMA, INC.","MIRATI THERAPEUTICS, INC.","KEMPHARM, INC","HERON THERAPEUTICS, INC. /DE/",ALEXION PHARMACEUTICALS INC,XOMA CORP,JOHNSON & JOHNSON,TREVENA INC,IMMUNOGEN INC,"ZYNERBA PHARMACEUTICALS, INC.",...,RIGEL PHARMACEUTICALS INC,IONIS PHARMACEUTICALS INC,PROGENICS PHARMACEUTICALS INC,EVOKE PHARMA INC,AKORN INC,"SIERRA ONCOLOGY, INC.","FIBROCELL SCIENCE, INC.",WAVE LIFE SCIENCES LTD.,LILLY ELI & CO,OPHTHOTECH CORP.
name,,,,,,,,,,,,,,,,,,,,,
"RECRO PHARMA, INC.",0.022964,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
"MIRATI THERAPEUTICS, INC.",0.000000,0.097249,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
"KEMPHARM, INC",0.000000,0.000000,0.056323,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
"HERON THERAPEUTICS, INC. /DE/",0.000000,0.000000,0.000000,0.01486,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
ALEXION PHARMACEUTICALS INC,0.000000,0.000000,0.000000,0.00000,0.012712,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"SIERRA ONCOLOGY, INC.",0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.049597,0.000000,0.000000,0.000000,0.000000
"FIBROCELL SCIENCE, INC.",0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.067369,0.000000,0.000000,0.000000
WAVE LIFE SCIENCES LTD.,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.037018,0.000000,0.000000


#### Efficient Frontier - Pharmaceutical Preparations

In [24]:
ef1 = EfficientFrontier(mu_Pharm, cov_Factor_Model, weight_bounds=(0, 0.2))

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef1, ax=ax, show_assets=True)

# Find and plot the tangency portfolio
ef2 = EfficientFrontier(mu_Pharm, cov_Factor_Model, weight_bounds=(0, 0.2))
# min volatility
ef2.min_volatility()
ret_tangent, std_tangent, _ = ef2.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Min Volatility")

# Format
ax.set_title("Efficient Frontier - Pharmaceutical Preparations \n Factor Model Estimates")
ax.legend()
plt.tight_layout()
plt.savefig('images/Efficient_Frontier_Returns.png', dpi=200, bbox_inches='tight')
plt.show()

NameError: name 'EfficientFrontier' is not defined

##### Min Volatility Portfolio

###### Performance

In [25]:
ef2.portfolio_performance(verbose=True);

NameError: name 'ef2' is not defined

###### Weights

In [26]:
companies = []
weights = []
for company, weight in ef2.clean_weights().items():
    if weight != 0:
        companies.append(company)
        weights.append(weight)
        
dic = {'Company_Name':companies,'Weight':weights}
min_vol = pd.DataFrame(dic)

NameError: name 'ef2' is not defined

In [27]:
companies = []
weights = []
for company, weight in ef2.clean_weights().items():
    if weight != 0:
        companies.append(company)
        weights.append(weight)
        
dic = {'Company_Name':companies,'Weight':weights}
min_vol = pd.DataFrame(dic)

NameError: name 'ef2' is not defined

In [28]:
min_vol.to_csv("data/min_vol_factor_model_Pharmaceutical_Preparations.csv", index = False)

NameError: name 'min_vol' is not defined

In [29]:
pd.read_csv("data/min_vol_factor_model_Pharmaceutical_Preparations.csv")

,Company_Name,Weight
0,ARENA PHARMACEUTICALS INC,0.07682
1,ANI PHARMACEUTICALS INC,0.05767
2,PROGENICS PHARMACEUTICALS INC,0.07063
3,JOHNSON & JOHNSON,0.10600
4,"PACIRA PHARMACEUTICALS, INC.",0.02880
5,XOMA CORP,0.04907
6,FLEXION THERAPEUTICS INC,0.06494
7,AXOVANT SCIENCES LTD.,0.02162
8,"TELIGENT, INC.",0.08773
9,"FORTRESS BIOTECH, INC.",0.00223


### Results for the Other 4 Industries

In [30]:
SIC_list = [7372, 1311, 6798, 6022]
SIC_desc = ['Prepackaged Software (mass reproduction of software)', 'Crude Petroleum and Natural Gas', 
           'Real Estate Investment Trusts', 'State Commercial Banks (commercial banking)']

#### Prepackaged Software (mass reproduction of software)

In [31]:
SIC = SIC_list[0]
    
industry_name = SIC_desc[SIC_list.index(SIC)]
    
# get the names of the companies in the other industries
Companies = df[df.SIC == SIC]
Company_list = Companies.index

# get the companies name that match return data and business description data
SET = (set(Company_list) & set(r_selected.index))
LIST = [*SET, ]

B_matrix = prob.T[LIST].T
B_matrix = B_matrix[~B_matrix.index.duplicated(keep="first")]

r = r_selected.T[LIST].T

coef_mat = pd.DataFrame(0, index = r.columns, columns = range(n_components))

LR = LinearRegression()
for i in range(len(r.columns)):
    date = r.columns[i]
    r_t_i = r[date] 
    reg = LR.fit(B_matrix, r_t_i)
    coef_mat.iloc[i] = reg.coef_

prediction = B_matrix.dot(coef_mat.T)
residual = r - prediction

mu_sample = mu[LIST]

cov_Factor_Model = pd.DataFrame(np.array(B_matrix.dot(coef_mat.cov()).dot(B_matrix.T)) + np.diag(np.diag(residual.T.cov()))).set_index(B_matrix.index)
cov_Factor_Model.columns = B_matrix.index
cov_Factor_Model
    
# perform minimum variance analysis
ef1 = EfficientFrontier(mu_sample, cov_Factor_Model, weight_bounds=(0, 0.2))

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef1, ax=ax, show_assets=True)

# Find and plot the tangency portfolio
ef2 = EfficientFrontier(mu_sample, cov_Factor_Model, weight_bounds=(0, 0.2))
# min volatility
ef2.min_volatility()
ret_tangent, std_tangent, _ = ef2.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Min Volatility")

# Format
ax.set_title("Efficient Frontier - %s \n Factor Model Estimates" %industry_name)
ax.legend()
plt.tight_layout()
plt.savefig('images/Efficient_Frontier_Factor_Model_Estimates' + str(industry_name) + '.png', dpi=200, bbox_inches='tight')
plt.show()

NameError: name 'EfficientFrontier' is not defined

##### Min Volatility Portfolio

###### Performance

In [32]:
ef2.portfolio_performance(verbose=True);

NameError: name 'ef2' is not defined

###### Weights

In [33]:
companies = []
weights = []
for company, weight in ef2.clean_weights().items():
    if weight != 0:
        companies.append(company)
        weights.append(weight)
        
dic = {'Company_Name':companies,'Weight':weights}
min_vol = pd.DataFrame(dic)
min_vol.to_csv("data/min_vol_factor_model_Prepackaged_Software.csv", index = False)

NameError: name 'ef2' is not defined

In [34]:
pd.read_csv("data/min_vol_factor_model_Prepackaged_Software.csv")

,Company_Name,Weight
0,APPFOLIO INC,0.02000
1,ELLIE MAE INC,0.00827
2,REALPAGE INC,0.02637
3,3D SYSTEMS CORP,0.02612
4,"POLARITYTE, INC.",0.20000
5,GSE SYSTEMS INC,0.01215
6,"2U, INC.",0.17282
7,VEEVA SYSTEMS INC,0.00393
8,"BLACK KNIGHT, INC.",0.01090
9,MICROSTRATEGY INC,0.12752


#### Crude Petroleum and Natural Gas
When we conduct the same analysis, there is no weight shown. Efficient frontier cannot be found.

In [35]:
SIC = SIC_list[1]
    
industry_name = SIC_desc[SIC_list.index(SIC)]
    
# get the names of the companies in the other industries
Companies = df[df.SIC == SIC]
Company_list = Companies.index

# get the companies name that match return data and business description data
SET = (set(Company_list) & set(r_selected.index))
LIST = [*SET, ]

mu_sample = mu[LIST]
# get the outliers
outlier = mu_sample[mu_sample>1].index
mu_sample = mu_sample.drop(outlier)
LIST = mu_sample.index

B_matrix = prob.T[LIST].T
B_matrix = B_matrix[~B_matrix.index.duplicated(keep="first")]

r = r_selected.T[LIST].T

coef_mat = pd.DataFrame(0, index = r.columns, columns = range(n_components))

LR = LinearRegression()

for i in range(len(r.columns)):
    date = r.columns[i]
    r_t_i = r[date] 
    reg = LR.fit(B_matrix, r_t_i)
    coef_mat.iloc[i] = reg.coef_

prediction = B_matrix.dot(coef_mat.T)
residual = r - prediction

cov_Factor_Model = pd.DataFrame(np.array(B_matrix.dot(coef_mat.cov()).dot(B_matrix.T)) + np.diag(np.diag(residual.T.cov()))).set_index(B_matrix.index)
cov_Factor_Model.columns = B_matrix.index
cov_Factor_Model
    
# perform minimum variance analysis
ef1 = EfficientFrontier(mu_sample, cov_Factor_Model, weight_bounds=(0, 0.2))

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef1, ax=ax, show_assets=True)

# Find and plot the tangency portfolio
ef2 = EfficientFrontier(mu_sample, cov_Factor_Model, weight_bounds=(0, 0.2))
# min volatility
ef2.min_volatility()
ret_tangent, std_tangent, _ = ef2.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Min Volatility")

# Format
ax.set_title("Efficient Frontier - %s \n Factor Model Estimates" %industry_name)
ax.legend()
plt.tight_layout()
plt.savefig('images/Efficient_Frontier_Factor_Model_Estimates' + str(industry_name) + '.png', dpi=200, bbox_inches='tight')
plt.show()

NameError: name 'EfficientFrontier' is not defined

##### Min Volatility Portfolio

###### Performance

In [36]:
ef2.portfolio_performance(verbose=True);

NameError: name 'ef2' is not defined

###### Weights

In [37]:
companies = []
weights = []
for company, weight in ef2.clean_weights().items():
    if weight != 0:
        companies.append(company)
        weights.append(weight)
        
dic = {'Company_Name':companies,'Weight':weights}
min_vol = pd.DataFrame(dic)
min_vol.to_csv("data/min_vol_factor_model_Crude_Petroleum_and_Natural_Gas.csv", index = False)

NameError: name 'ef2' is not defined

In [38]:
pd.read_csv("data/min_vol_factor_model_Crude_Petroleum_and_Natural_Gas.csv")

,Company_Name,Weight
0,GULFPORT ENERGY CORP,0.01520
1,NOBLE ENERGY INC,0.00010
2,ANTERO RESOURCES CORP,0.07803
3,WHITING PETROLEUM CORP,0.05473
4,ISRAMCO INC,0.04350
5,CARRIZO OIL & GAS INC,0.03058
6,CIMAREX ENERGY CO,0.00453
7,KOSMOS ENERGY LTD.,0.12827
8,SRC ENERGY INC.,0.01361
9,PEDEVCO CORP,0.05452


#### Real Estate Investment Trusts

In [39]:
SIC = SIC_list[2]
    
industry_name = SIC_desc[SIC_list.index(SIC)]
    
# get the names of the companies in the other industries
Companies = df[df.SIC == SIC]
Company_list = Companies.index

# get the companies name that match return data and business description data
SET = (set(Company_list) & set(r_selected.index))
LIST = [*SET, ]

mu_sample = mu[LIST]
# get the outliers
outlier = mu_sample[mu_sample>1].index
mu_sample = mu_sample.drop(outlier)
LIST = mu_sample.index

B_matrix = prob.T[LIST].T
B_matrix = B_matrix[~B_matrix.index.duplicated(keep="first")]

r = r_selected.T[LIST].T

coef_mat = pd.DataFrame(0, index = r.columns, columns = range(n_components))

LR = LinearRegression()

for i in range(len(r.columns)):
    date = r.columns[i]
    r_t_i = r[date] 
    reg = LR.fit(B_matrix, r_t_i)
    coef_mat.iloc[i] = reg.coef_

prediction = B_matrix.dot(coef_mat.T)
residual = r - prediction

cov_Factor_Model = pd.DataFrame(np.array(B_matrix.dot(coef_mat.cov()).dot(B_matrix.T)) + np.diag(np.diag(residual.T.cov()))).set_index(B_matrix.index)
cov_Factor_Model.columns = B_matrix.index
cov_Factor_Model
    
# perform minimum variance analysis
ef1 = EfficientFrontier(mu_sample, cov_Factor_Model, weight_bounds=(0, 0.2))

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef1, ax=ax, show_assets=True)

# Find and plot the tangency portfolio
ef2 = EfficientFrontier(mu_sample, cov_Factor_Model, weight_bounds=(0, 0.2))
# min volatility
ef2.min_volatility()
ret_tangent, std_tangent, _ = ef2.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Min Volatility")

# Format
ax.set_title("Efficient Frontier - %s \n Factor Model Estimates" %industry_name)
ax.legend()
plt.tight_layout()
plt.savefig('images/Efficient_Frontier_Factor_Model_Estimates' + str(industry_name) + '.png', dpi=200, bbox_inches='tight')
plt.show()

NameError: name 'EfficientFrontier' is not defined

##### Min Volatility Portfolio

###### Performance

In [40]:
ef2.portfolio_performance(verbose=True);

NameError: name 'ef2' is not defined

###### Weights

In [41]:
companies = []
weights = []
for company, weight in ef2.clean_weights().items():
    if weight != 0:
        companies.append(company)
        weights.append(weight)
        
dic = {'Company_Name':companies,'Weight':weights}
min_vol = pd.DataFrame(dic)
min_vol.to_csv("data/min_vol_factor_model_Real_Estate_Investment_Trusts.csv", index = False)

NameError: name 'ef2' is not defined

In [42]:
pd.read_csv("data/min_vol_factor_model_Real_Estate_Investment_Trusts.csv")

,Company_Name,Weight
0,VORNADO REALTY TRUST,0.01771
1,KILROY REALTY CORP,0.07864
2,OUTFRONT MEDIA INC.,0.07300
3,EQUITY RESIDENTIAL,0.12677
4,"EMPIRE STATE REALTY TRUST, INC.",0.03364
5,HOWARD HUGHES CORP,0.08041
6,CYRUSONE INC.,0.08523
7,ISTAR INC.,0.02747
8,FRANKLIN STREET PROPERTIES CORP /MA/,0.04327
9,CHERRY HILL MORTGAGE INVESTMENT CORP,0.03513


#### State Commercial Banks (commercial banking)

In [43]:
SIC = SIC_list[3]
    
industry_name = SIC_desc[SIC_list.index(SIC)]
    
# get the names of the companies in the other industries
Companies = df[df.SIC == SIC]
Company_list = Companies.index

# get the companies name that match return data and business description data
SET = (set(Company_list) & set(r_selected.index))
LIST = [*SET, ]

B_matrix = prob.T[LIST].T
B_matrix = B_matrix[~B_matrix.index.duplicated(keep="first")]

r = r_selected.T[LIST].T

coef_mat = pd.DataFrame(0, index = r.columns, columns = range(n_components))

LR = LinearRegression()

for i in range(len(r.columns)):
    date = r.columns[i]
    r_t_i = r[date] 
    reg = LR.fit(B_matrix, r_t_i)
    coef_mat.iloc[i] = reg.coef_

prediction = B_matrix.dot(coef_mat.T)
residual = r - prediction

mu_sample = mu[LIST]

cov_Factor_Model = pd.DataFrame(np.array(B_matrix.dot(coef_mat.cov()).dot(B_matrix.T)) + np.diag(np.diag(residual.T.cov()))).set_index(B_matrix.index)
cov_Factor_Model.columns = B_matrix.index
cov_Factor_Model


# perform minimum variance analysis
ef1 = EfficientFrontier(mu_sample, cov_Factor_Model, weight_bounds=(0, 0.2))


fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef1, ax=ax, show_assets=True)

# Find and plot the tangency portfolio
ef2 = EfficientFrontier(mu_sample, cov_Factor_Model, weight_bounds=(0, 0.2))
# min volatility
ef2.min_volatility()
ret_tangent, std_tangent, _ = ef2.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Min Volatility")

# Format
ax.set_title("Efficient Frontier - %s \n Factor Model Estimates" %industry_name)
ax.legend()
plt.tight_layout()
plt.savefig('images/Efficient_Frontier_Factor_Model_Estimates' + str(industry_name) + '.png', dpi=200, bbox_inches='tight')
plt.show()

NameError: name 'EfficientFrontier' is not defined

##### Min Volatility Portfolio

###### Performance

In [44]:
ef2.portfolio_performance(verbose=True);

NameError: name 'ef2' is not defined

###### Weights

In [45]:
companies = []
weights = []
for company, weight in ef2.clean_weights().items():
    if weight != 0:
        companies.append(company)
        weights.append(weight)
        
dic = {'Company_Name':companies,'Weight':weights}
min_vol = pd.DataFrame(dic)
min_vol.to_csv("data/min_vol_factor_model_State_Commercial_Banks.csv", index = False)

NameError: name 'ef2' is not defined

In [46]:
pd.read_csv("data/min_vol_factor_model_State_Commercial_Banks.csv")

,Company_Name,Weight
0,ENTERPRISE FINANCIAL SERVICES CORP,0.01757
1,"SERVISFIRST BANCSHARES, INC.",0.03152
2,WESTERN ALLIANCE BANCORPORATION,0.01272
3,BAR HARBOR BANKSHARES,0.00804
4,FIRST COMMUNITY CORP /SC/,0.01502
5,VILLAGE BANK & TRUST FINANCIAL CORP.,0.01429
6,"RELIANT BANCORP, INC.",0.00914
7,M&T BANK CORP,0.02439
8,WEST BANCORPORATION INC,0.02403
9,CITIZENS FINANCIAL GROUP INC/RI,0.00108
